In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import plotly.colors
import wbdata
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
energy_data = pd.read_csv('../Data/AnalysisData.csv')
energy_data

,Unnamed: 0,country,year,iso_code,gdp,population,biofuel_consumption,biofuel_electricity,coal_consumption,coal_electricity,...,per_capita_electricity,primary_energy_consumption,renewables_consumption,renewables_electricity,solar_consumption,solar_electricity,wind_consumption,wind_electricity,GNI,Socio-Economic Status
0,0,Afghanistan,2000,AFG,1.128379e+10,19542986.0,0.0,0.00,0.0,0.00,...,24.050,5.914,0.0,0.31,0.0,0.00,0.0,0.0,NaN,NaN
1,1,Afghanistan,2001,AFG,1.102127e+10,19688634.0,0.0,0.00,0.0,0.00,...,29.967,4.664,0.0,0.50,0.0,0.00,0.0,0.0,NaN,NaN
2,2,Afghanistan,2002,AFG,1.880487e+10,21000258.0,0.0,0.00,0.0,0.00,...,32.857,4.428,0.0,0.56,0.0,0.00,0.0,0.0,180.0,Low-income
3,3,Afghanistan,2003,AFG,2.107434e+10,22645136.0,0.0,0.00,0.0,0.00,...,41.510,5.208,0.0,0.63,0.0,0.00,0.0,0.0,190.0,Low-income
4,4,Afghanistan,2004,AFG,2.233257e+10,23553554.0,0.0,0.00,0.0,0.00,...,37.786,4.810,0.0,0.56,0.0,0.00,0.0,0.0,210.0,Low-income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4901,4901,Zimbabwe,2018,ZWE,2.271535e+10,15052191.0,0.0,0.39,0.0,3.69,...,610.542,51.809,0.0,5.46,0.0,0.02,0.0,0.0,2140.0,Lower-middle-income
4902,4902,Zimbabwe,2019,ZWE,0.000000e+00,15354606.0,0.0,0.38,0.0,3.62,...,536.647,46.120,0.0,4.58,0.0,0.03,0.0,0.0,NaN,NaN
4903,4903,Zimbabwe,2020,ZWE,0.000000e+00,15669663.0,0.0,0.35,0.0,3.36,...,484.375,41.997,0.0,4.19,0.0,0.03,0.0,0.0,1690.0,Lower-middle-income
4904,4904,Zimbabwe,2021,ZWE,0.000000e+00,15993525.0,0.0,0.38,0.0,3.57,...,502.078,42.145,0.0,4.42,0.0,0.04,0.0,0.0,NaN,NaN


In [3]:
energy_data = energy_data.drop(columns='Unnamed: 0')
energy_data['non_renewable_consumption'] = energy_data[['coal_consumption', 'fossil_fuel_consumption','gas_consumption', 'nuclear_consumption', 'oil_consumption']].sum(axis=1)
energy_data.head()

,country,year,iso_code,gdp,population,biofuel_consumption,biofuel_electricity,coal_consumption,coal_electricity,electricity_demand,...,primary_energy_consumption,renewables_consumption,renewables_electricity,solar_consumption,solar_electricity,wind_consumption,wind_electricity,GNI,Socio-Economic Status,non_renewable_consumption
0,Afghanistan,2000,AFG,1.128379e+10,19542986.0,0.0,0.0,0.0,0.0,0.57,...,5.914,0.0,0.31,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1,Afghanistan,2001,AFG,1.102127e+10,19688634.0,0.0,0.0,0.0,0.0,0.69,...,4.664,0.0,0.50,0.0,0.0,0.0,0.0,NaN,NaN,0.0
2,Afghanistan,2002,AFG,1.880487e+10,21000258.0,0.0,0.0,0.0,0.0,0.79,...,4.428,0.0,0.56,0.0,0.0,0.0,0.0,180.0,Low-income,0.0
3,Afghanistan,2003,AFG,2.107434e+10,22645136.0,0.0,0.0,0.0,0.0,1.04,...,5.208,0.0,0.63,0.0,0.0,0.0,0.0,190.0,Low-income,0.0
4,Afghanistan,2004,AFG,2.233257e+10,23553554.0,0.0,0.0,0.0,0.0,0.99,...,4.810,0.0,0.56,0.0,0.0,0.0,0.0,210.0,Low-income,0.0


In [4]:
# Predict the Renewable Energy Consumption for the year 2022
# Predictors Used: 'year', 'iso_code', 'gdp', 'population', 'GNI'
# Output: 'renewables_consumption'

train = energy_data[~energy_data['year'].isin([2021, 2022])]
test = energy_data[energy_data['year'].isin([2021, 2022])]

# Convert string columns to categorical dummies (factors)
train = pd.get_dummies(train, columns=['iso_code'], drop_first=True)

test = pd.get_dummies(test, columns=['iso_code'], drop_first=True)
train['gdp_per_capita'] = train['gdp']/train['population']
test['gdp_per_capita'] = test['gdp']/test['population']

col_list = train.columns
model_list = ['year','gdp_per_capita','population', 'GNI']
for col in col_list:
    name = col.split('_')

    for val in name:
        if val=='code':
            model_list.append(col)
#print(model_list)    
        
# Create model
model = LinearRegression()
# Fit model
model.fit(train[model_list].fillna(0), train['renewables_consumption'])
predictions = model.predict(test[model_list].fillna(0))

print("R² Score:", r2_score(test['renewables_consumption'], predictions))
#print("MSE:", mean_squared_error(test['renewables_consumption'], predictions))

R² Score: 0.7325016936218002


In [6]:
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42) # You can adjust hyperparameters
rf_regressor.fit(train[model_list].fillna(0), train['renewables_consumption'])
y_pred_reg = rf_regressor.predict(test[model_list].fillna(0))
print(f"R² Score: {r2_score(test['renewables_consumption'], y_pred_reg)}")

R² Score: 0.8017487682565398


In [7]:
test_df = pd.read_csv('../Data/testData.csv')
df_test = test_df.copy()
df_test= pd.get_dummies(df_test, columns=['iso_code'], drop_first=True)
#test_df['predicted_renewables_consumption'] = rf_regressor.predict(df_test[model_list].fillna(0))
#test_df.head()
col_list = df_test.columns
test_model_list = ['year','gdp','population', 'GNI']
for col in col_list:
    name = col.split('_')

    for val in name:
        if val=='code':
            test_model_list.append(col)
missing = [x for x in model_list if x not in test_model_list]
for col in missing:
    df_test[col] = False
df_test['gdp_per_capita'] = df_test['gdp per capita nominal']
test_df['predicted_renewables_consumption'] = rf_regressor.predict(df_test[model_list].fillna(0))
test_df.head()

,Unnamed: 0,country,year,population,gdp per capita nominal,gdp per capita ppp,iso_code,GNI,predicted_renewables_consumption
0,0,Afghanistan,2023,41454761,410.933,2173.745,AFG,380.0,0.14076
1,1,Albania,2024,2791765,9598.191,21376.586,ALB,NaN,0.00000
2,2,Albania,2023,2811655,8299.278,20018.303,ALB,7680.0,0.00000
3,3,Algeria,2024,46814308,5579.128,17718.286,DZA,NaN,7.33403
4,4,Algeria,2023,46164219,5221.813,16900.136,DZA,4950.0,10.16934


In [9]:
test_df.to_csv("../Data/PredictedRenewableConsumption.csv")